<a href="https://colab.research.google.com/github/divgup/cs671/blob/master/assignment1_q2(mnist_data).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from tensorflow.keras.datasets import mnist
from keras.utils import to_categorical
#X_train_orig is a 3-D array containing the pixel values of grey image
#Y_train_orig is 1-D array containing labels of corresponding image 
#similarily for X_test and Y_test 
(X_train_orig,Y_train_orig),(X_test_orig,Y_test_orig) = mnist.load_data()
#reshape to a column vector
print(Y_train_orig.shape)
Y_tr_resh = Y_train_orig.reshape(60000,1)
Y_te_resh = Y_test_orig.reshape(10000,1)

#one hot encoding to give a 2-D array of size (60000,10) and (10000,10) for training and testing set resp.
Y_tr_T = to_categorical(Y_tr_resh,num_classes=10)
Y_te_T = to_categorical(Y_te_resh,num_classes=10)
Y_train = Y_tr_T.T
Y_test = Y_te_T.T
print(Y_train.shape)


Using TensorFlow backend.


11493376/11490434 [==============================] - 0s 0us/step
(60000,)
(10, 60000)


In [2]:
X_train_flat = X_train_orig.reshape(X_train_orig.shape[0],-1).T
X_test_flat = X_test_orig.reshape(X_test_orig.shape[0],-1).T
X_train = X_train_flat/255.
X_test = X_test_flat/255.
print(X_train_flat.shape)

(784, 60000)


In [0]:
def softmax(a):
  return np.exp(a)/(np.sum(np.exp(a),axis=0,keepdims=True))
def Relu(u):
  u[u<0] = 0
  return u  

In [0]:
parameters = {}
def initialisation():
  #hardcoded the no. of units in each layer
  layers = [784,50,50,10]
  for it in range(1,len(layers)):
    parameters["w"+str(it)] = np.random.rand(layers[it-1],layers[it])*np.sqrt(2/layers[it-1])
    parameters["b"+str(it)] = np.zeros((layers[it],1))
  #print(parameters["w2"].shape)
  #print(weights[1])


In [0]:
initialisation()
print(parameters)

In [0]:
def calc_loss(Y_train,output):
  loss = np.sum(np.sum(-Y_train*np.log(output),axis=0,keepdims=True),axis=1,keepdims=True)
  return loss/Y_train.shape[1]

In [0]:
layer = {}
def forward_pass(x,num_layers):
  for it in range(num_layers-1):
    #print(parameters["w"+str(it+1)].shape)
    layer["pre_hid"+str(it+1)] = np.dot(parameters["w"+str(it+1)].T,x)+parameters["b"+str(it+1)]
    #print(layer)
    if it < num_layers-2:
      layer["activated"+str(it+1)] = Relu(layer["pre_hid"+str(it+1)])
    else:
      #print(layer["pre_hid"+str(it+1)])
      layer["activated"+str(it+1)] = softmax(layer["pre_hid"+str(it+1)])
      #print(layer["activated"+str(it+1)])
    x = layer["pre_hid"+str(it+1)]    
    #if it < len(layers)-2):
      #x = Relu(l)
    #else:
      #x = softmax(l)
  #return layer      


In [0]:
def drelu(x):
  y = np.ones(x.shape)
  y[x<0] = 0
  return y

In [0]:
def back_prop(X_train,num_layers):
  output = layer["activated"+str(num_layers-1)]
  #print(output.shape)
  #print(Y_train.shape)
  grad = {}
  m = X_train.shape[1]
  grad["a"+str(num_layers-1)] = -(Y_train-output)/m
  grad["h"+str(num_layers-2)] = np.dot(parameters["w"+str(num_layers-1)],grad["a"+str(num_layers-1)])
  #grad["w"+str(len(layers)-1)] = np.dot(output-Y_train,x[pre_hid+str(len(layers)-2)].T)
  for it in range(1,num_layers-1):
    Indicator = drelu(layer["pre_hid"+str(num_layers-it-1)])
    grad["a"+str(num_layers-1-it)] = np.multiply(grad["h"+str(num_layers-it-1)],Indicator) 
    grad["h"+str(num_layers-it-2)] = np.dot(parameters["w"+str(num_layers-it-1)],grad["a"+str(num_layers-1-it)])
  grad["w1"] = np.dot(grad["a1"],X_train.T)
  grad["b1"] = np.sum(grad["a"+str(1)],axis=1,keepdims=True)
  for it in range(1,num_layers-1):  
    grad["w"+str(it+1)] = np.dot(grad["a"+str(it+1)],layer["pre_hid"+str(it)].T) 
    grad["b"+str(it+1)] = np.sum(grad["a"+str(it+1)],axis=1,keepdims=True)
  return grad  
  #del_loss_resp_to_h_l0 = np.dot(parameter["w2"],-(Y_train-output))
  #pre_hid2 = x["pre_hid"+str(len(layers)-2)]
  #i = np.ones(pre_hid2.shape)
  #pre_hid1 = x["pre_hid"+str(len(layers)-3)] 
  #i[pre_hid2<0] = 0
  #del_loss_resp_to_al = np.multiply(del_loss_resp_to_h_l0,i)
  #del_loss_resp_to_h_l1 = np.dot(paramter["w1"],del_loss_resp_to_al)

In [0]:
def weight_update(num_layers,alpha,grad):
  for i in range(num_layers-1):
    parameters["w"+str(num_layers-i-1)] = parameters["w"+str(num_layers-i-1)] - (alpha*grad["w"+str(num_layers-i-1)]).T
    parameters["b"+str(num_layers-i-1)] = parameters["b"+str(num_layers-i-1)] - (alpha*grad["b"+str(num_layers-i-1)])
  return parameters  

In [15]:
alpha = 0.01
num_layers = 4
cost=[]
num_itr = 3000
num_hidden_layers = 2
initialisation()
forward_pass(X_train,num_layers)

#print(layer["activated"+str(num_layers-1)])
for it in range(num_itr):
  forward_pass(X_train,num_layers)
  output = layer["activated"+str(num_layers-1)]
  grad = back_prop(X_train,num_layers)
  #print(parameters["b1"].shape)
  weight_update(num_layers,alpha,grad)
  if it%50==0:
    cost.append(calc_loss(Y_train,output))
    print(calc_loss(Y_train,output))
  #print(parameters["b1"].shape)    

[[7.20441103]]
[[2.82638553]]
[[2.34099539]]
[[2.30852984]]
[[2.28095241]]
[[2.2093381]]
[[2.17421669]]
[[2.1193431]]
[[2.04853542]]
[[1.95537801]]
[[1.83449687]]
[[1.68444129]]
[[1.53312531]]
[[1.39205481]]
[[1.26827654]]
[[1.16471807]]
[[1.07944335]]
[[1.00884437]]
[[0.9493274]]
[[0.89827458]]


In [16]:
#print(parameters)
output = layer["activated"+str(num_layers-1)]
acc = np.sum(np.sum(Y_train*output,axis=0,keepdims=True),axis=1,keepdims=True)/Y_train.shape[1]
#def metrics():
acc_percentage = acc*100
print(acc_percentage)

[[50.67899838]]
